In [281]:
import numpy as np
import plotly
import pandas as pd
import config
import os
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [282]:
def cleanData(path):
    df = pd.read_csv(path)
    
    df.assign(count = 1)
    df['price_sqm'] = round(df['price'] / df['sqm'],0)
    df_original=df.copy()

    df['rooms']=df['rooms'].replace('na', np.nan).copy()
    df['toilets']=df['toilets'].replace('na', np.nan).copy()
    df = df.dropna(axis='index', how='any').copy()

    df['rooms'] = df['rooms'].astype(str).apply(lambda x: pd.to_numeric(x.replace('+','')) +1 )
    df['toilets'] = df['toilets'].astype(str).apply(lambda x: pd.to_numeric(x.replace('+','')) +1 )

    df['posting_title'] = df['posting_title'].astype('string')
    df['link'] = df['link'].astype('string')
    df['location'] = df['location'].astype('string')
    df['link'] = df['link'].astype('string')
    df['city'] = df['city'].astype('string')
    df['posting_title'] = df['posting_title'].astype('string')

    df['sqm'] =  df['sqm'].astype(int)
    df['rooms']=df['rooms'].astype(int)
    df['toilets']=df['toilets'].astype(int)
    label_map = {
                'mini': lambda x: x < 50,
                'small': lambda x: x < 75,
                'regular': lambda x: x < 100,
                'big': lambda x: x < 150,
                'huge': lambda x: x >= 150
                }
    df['size_cluser'] = df['sqm'].apply(lambda x: next((label for label, condition in label_map.items() if condition(x)), None))
    return df , df_original 
    

In [283]:
def drop_outliers(df):
    df_means = df.groupby(['location','size_cluser'])['price_sqm'].mean().reset_index()
    print(df_means)
    df = pd.merge(df , df_means , on=['location', 'size_cluster'] , how='left')
    return df , df_means

## Clean Data

In [284]:
df_rent , df_rent_original  = cleanData("/home/crd/Portfolio_Repo-1/Output/Database_Rent.csv")
df_sales , df_sales_original = cleanData("/home/crd/Portfolio_Repo-1/Output/Database_Sales.csv")

## Drop Outliers

In [285]:
df_rent , df_means = drop_outliers(df_rent)
df_means

KeyError: 'size_cluster'

In [ ]:
df['price_sqm_range'] = pd.qcut(df['price_sqm'], 4 , labels=['lowest', 'low', 'medium','high'])
df_sqmprice_location = df.drop(['posting_title', 'price', 'rooms', 'sqm', 'toilets', 'link', 'city','price_sqm_range'] , axis=1)

df_sqmprice_location=df_sqmprice_location.groupby(by='location').agg({'price_sqm': ['mean', 'min', 'max', 'count']}).astype(int)


In [ ]:
df_sqmprice_location=df_sqmprice_location.sort_values(by=('price_sqm', 'mean'),ascending=False)
df_sqmprice_location[:5]

price_sqm              
                                           mean min max count
location                                                     
Centro Storico                               27   5  58   292
Testaccio, Trastevere                        26  16  58    67
Aventino, San Saba, Caracalla                23  14  47    37
Prati, Borgo, Mazzini, Delle Vittorie        22   6  60   111
Termini, Repubblica                          21  12  56    35